# .NET Semantic Kernel Workshop

This notebook demonstrates how to build AI agents using Microsoft Semantic Kernel in .NET 9 with Azure AI services.

## Prerequisites
- .NET 9 SDK installed
- .NET Interactive extension for Jupyter
- Azure OpenAI or Azure AI Inference endpoint
- Valid environment configuration

## Workshop Overview
1. Install .NET Interactive Extension
2. Set Up .NET Environment
3. Create Basic .NET Project Structure
4. Generate Notebook Metadata
5. Create Notebook Cells with .NET Code
6. Write Notebook to File
7. Validate Generated Notebook

## 1. Install .NET Interactive Extension

First, ensure you have the .NET Interactive extension installed. This enables running C# code in Jupyter notebooks.

In [ ]:
// Check if .NET Interactive is available
Console.WriteLine($".NET Version: {Environment.Version}");
Console.WriteLine($"Runtime: {System.Runtime.InteropServices.RuntimeInformation.FrameworkDescription}");
Console.WriteLine("✅ .NET Interactive is working!");

## 2. Set Up .NET Environment

Import necessary packages and set up the environment for working with Semantic Kernel and Azure AI services.

In [ ]:
// Install required NuGet packages
#r "nuget: Microsoft.SemanticKernel, 1.30.0"
#r "nuget: Microsoft.SemanticKernel.Agents.OpenAI, 1.30.0-alpha"
#r "nuget: Azure.AI.Inference, 1.0.0-beta.1"
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"
#r "nuget: Microsoft.Extensions.DependencyInjection, 8.0.0"
#r "nuget: Microsoft.Extensions.Logging, 8.0.0"

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.OpenAI;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Azure.AI.Inference;
using Microsoft.Extensions.Configuration;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.Extensions.Logging;
using System.ComponentModel;
using System.Text.Json;
using System.Text.Json.Serialization;

Console.WriteLine("✅ Packages and namespaces imported successfully!");

## 3. Create Basic .NET Project Structure

Define the basic structure and configuration for our Semantic Kernel project.

In [ ]:
// Configuration class for Azure AI services
public class AzureAIConfig
{
    public string EndpointType { get; set; } = "AzureOpenAI";
    public string Endpoint { get; set; } = "";
    public string ApiKey { get; set; } = "";
    public string ModelDeploymentName { get; set; } = "gpt-4o";
    public string ApiVersion { get; set; } = "2024-08-01-preview";
}

// Load configuration from environment or appsettings
var config = new AzureAIConfig();

// Try to load from environment variables
config.Endpoint = Environment.GetEnvironmentVariable("AZURE_AI_ENDPOINT") ?? "";
config.ApiKey = Environment.GetEnvironmentVariable("AZURE_AI_API_KEY") ?? "";
config.ModelDeploymentName = Environment.GetEnvironmentVariable("AZURE_AI_MODEL_DEPLOYMENT_NAME") ?? "gpt-4o";

Console.WriteLine($"Configuration loaded - Endpoint Type: {config.EndpointType}");
Console.WriteLine($"Model: {config.ModelDeploymentName}");
Console.WriteLine($"Endpoint configured: {!string.IsNullOrEmpty(config.Endpoint)}");

## 4. Initialize Semantic Kernel

Set up the Semantic Kernel with Azure AI services using managed identity and secure authentication.

In [ ]:
// Initialize Semantic Kernel with Azure AI services
var builder = Kernel.CreateBuilder();

// Configure logging
using ILoggerFactory loggerFactory = LoggerFactory.Create(builder =>
    builder.AddConsole().SetMinimumLevel(LogLevel.Information));

try
{
    if (config.EndpointType == "AzureOpenAI")
    {
        // Azure OpenAI configuration with secure authentication
        builder.AddAzureOpenAIChatCompletion(
            deploymentName: config.ModelDeploymentName,
            endpoint: config.Endpoint,
            apiKey: config.ApiKey, // In production, use Managed Identity
            serviceId: "azure-openai",
            modelId: config.ModelDeploymentName
        );
        Console.WriteLine("✅ Azure OpenAI service configured");
    }
    else if (config.EndpointType == "AzureAIInference")
    {
        // Azure AI Inference configuration
        builder.AddAzureAIInferenceChatCompletion(
            modelId: config.ModelDeploymentName,
            endpoint: new Uri(config.Endpoint),
            credential: new System.ClientModel.ApiKeyCredential(config.ApiKey)
        );
        Console.WriteLine("✅ Azure AI Inference service configured");
    }

    // Build the kernel
    var kernel = builder.Build();
    
    // Test the connection
    var chatService = kernel.GetRequiredService<IChatCompletionService>();
    Console.WriteLine($"✅ Kernel initialized successfully with {config.EndpointType}");
    Console.WriteLine($"Chat service: {chatService.GetType().Name}");
}
catch (Exception ex)
{
    Console.WriteLine($"❌ Error initializing kernel: {ex.Message}");
    throw;
}

## 5. Create AI Agents

Define specialized AI agents for different roles using Semantic Kernel's agent framework.

In [ ]:
// Create specialized AI agents with different personalities and roles
public class AgentDefinition
{
    public string Name { get; set; }
    public string Instructions { get; set; }
    public string Description { get; set; }
}

// Define agent configurations
var agentDefinitions = new List<AgentDefinition>
{
    new AgentDefinition
    {
        Name = "TechExpert",
        Instructions = @"You are a technical expert specializing in software development, cloud computing, and AI technologies. 
                        Provide detailed, accurate technical information with practical examples. 
                        Always consider security, performance, and best practices in your responses.",
        Description = "Expert in software development and cloud technologies"
    },
    new AgentDefinition
    {
        Name = "BusinessAnalyst",
        Instructions = @"You are a business analyst who excels at understanding business requirements and translating them into technical specifications. 
                        Focus on ROI, user experience, and business value. 
                        Ask clarifying questions to better understand business needs.",
        Description = "Business analyst focused on requirements and value delivery"
    },
    new AgentDefinition
    {
        Name = "ProjectManager",
        Instructions = @"You are an experienced project manager skilled in agile methodologies and team coordination. 
                        Help break down complex tasks, estimate timelines, and identify potential risks. 
                        Focus on practical project execution and team collaboration.",
        Description = "Project manager specializing in agile delivery"
    }
};

Console.WriteLine($"✅ Defined {agentDefinitions.Count} agent configurations:");
foreach (var agent in agentDefinitions)
{
    Console.WriteLine($"  - {agent.Name}: {agent.Description}");
}

In [ ]:
// Create actual agent instances using OpenAI Assistant API
var agents = new Dictionary<string, ChatCompletionAgent>();

try
{
    foreach (var agentDef in agentDefinitions)
    {
        var agent = new ChatCompletionAgent()
        {
            Instructions = agentDef.Instructions,
            Name = agentDef.Name,
            Kernel = kernel,
            Arguments = new KernelArguments(new OpenAIPromptExecutionSettings()
            {
                Temperature = 0.7,
                MaxTokens = 1000,
                TopP = 1.0
            })
        };
        
        agents[agentDef.Name] = agent;
        Console.WriteLine($"✅ Created agent: {agentDef.Name}");
    }
    
    Console.WriteLine($"\n✅ Successfully created {agents.Count} AI agents");
}
catch (Exception ex)
{
    Console.WriteLine($"❌ Error creating agents: {ex.Message}");
    throw;
}

## 6. Test Single Agent Interaction

Test individual agent responses to verify they're working correctly.

In [ ]:
// Test individual agent interaction
async Task TestSingleAgent(string agentName, string message)
{
    if (!agents.ContainsKey(agentName))
    {
        Console.WriteLine($"❌ Agent '{agentName}' not found");
        return;
    }

    try
    {
        Console.WriteLine($"\n🤖 Testing {agentName} Agent");
        Console.WriteLine($"📝 Input: {message}");
        Console.WriteLine("🔄 Processing...\n");

        var agent = agents[agentName];
        var history = new ChatHistory();
        history.AddUserMessage(message);

        await foreach (var response in agent.InvokeStreamingAsync(history))
        {
            Console.Write(response.Content);
        }
        
        Console.WriteLine($"\n\n✅ {agentName} response completed\n");
    }
    catch (Exception ex)
    {
        Console.WriteLine($"❌ Error testing {agentName}: {ex.Message}");
    }
}

// Test the TechExpert agent
await TestSingleAgent("TechExpert", "What are the best practices for implementing microservices with .NET and Azure?");

## 7. Create Agent Group Chat

Implement a multi-agent conversation system where agents can collaborate and discuss topics.

In [ ]:
// Create an agent group chat with selection strategy
var agentGroupChat = new AgentGroupChat(agents.Values.ToArray())
{
    ExecutionSettings = new()
    {
        // Configure termination strategy
        TerminationStrategy = new KernelFunctionTerminationStrategy(
            KernelFunctionFactory.CreateFromMethod(
                ([Description("The chat history")] ChatHistory history) =>
                {
                    // Terminate after 6 messages or if conclusion is reached
                    return history.Count >= 6 || 
                           history.LastOrDefault()?.Content?.Contains("conclusion", StringComparison.OrdinalIgnoreCase) == true;
                }),
            kernel)
        {
            MaximumIterations = 10
        },
        
        // Configure agent selection strategy
        SelectionStrategy = new KernelFunctionSelectionStrategy(
            KernelFunctionFactory.CreateFromMethod(
                ([Description("The chat history")] ChatHistory history) =>
                {
                    // Simple round-robin selection with some intelligence
                    var messageCount = history.Count;
                    var agentNames = agents.Keys.ToArray();
                    
                    // First message always goes to TechExpert
                    if (messageCount <= 1) return "TechExpert";
                    
                    // Cycle through agents
                    var index = (messageCount - 1) % agentNames.Length;
                    return agentNames[index];
                }),
            kernel)
    }
};

Console.WriteLine("✅ Agent group chat configured with intelligent selection and termination strategies");

In [ ]:
// Execute a group chat conversation
async Task RunGroupChat(string initialMessage)
{
    try
    {
        Console.WriteLine("🚀 Starting Agent Group Chat");
        Console.WriteLine($"📝 Topic: {initialMessage}");
        Console.WriteLine("=" * 80);

        // Add initial user message
        agentGroupChat.AddChatMessage(new ChatMessageContent(AuthorRole.User, initialMessage));

        // Execute the group chat
        await foreach (var response in agentGroupChat.InvokeStreamingAsync())
        {
            Console.WriteLine($"\n🤖 {response.AuthorName ?? "System"}:");
            Console.WriteLine($"{response.Content}");
            Console.WriteLine("-" * 40);
        }

        Console.WriteLine("\n✅ Group chat completed");
    }
    catch (Exception ex)
    {
        Console.WriteLine($"❌ Error in group chat: {ex.Message}");
        Console.WriteLine($"Stack trace: {ex.StackTrace}");
    }
}

// Start a group discussion
await RunGroupChat("We need to build a new e-commerce platform using cloud-native technologies. What should we consider for architecture, timeline, and implementation approach?");

## 8. Advanced Features - Function Calling

Demonstrate how to add custom functions that agents can call to extend their capabilities.

In [ ]:
// Define custom functions that agents can call
public class BusinessFunctions
{
    [KernelFunction, Description("Calculate project cost estimation based on complexity and duration")]
    public string CalculateProjectCost(
        [Description("Project complexity: simple, medium, complex")] string complexity,
        [Description("Duration in months")] int durationMonths,
        [Description("Team size")] int teamSize)
    {
        var complexityMultiplier = complexity.ToLower() switch
        {
            "simple" => 1.0,
            "medium" => 1.5,
            "complex" => 2.0,
            _ => 1.5
        };

        var baseCostPerMonth = 15000; // Base cost per team member per month
        var totalCost = baseCostPerMonth * teamSize * durationMonths * complexityMultiplier;

        return $"Estimated project cost: ${totalCost:N0} " +
               $"(Complexity: {complexity}, Duration: {durationMonths} months, Team: {teamSize} people)";
    }

    [KernelFunction, Description("Get current technology trends and recommendations")]
    public string GetTechnologyTrends([Description("Technology domain")] string domain)
    {
        var trends = domain.ToLower() switch
        {
            "cloud" => "Serverless computing, Kubernetes, Multi-cloud strategies, Edge computing",
            "ai" => "Large Language Models, Computer Vision, MLOps, Responsible AI",
            "web" => "Progressive Web Apps, JAMstack, Micro-frontends, WebAssembly",
            "mobile" => "Cross-platform development, Flutter, React Native, 5G integration",
            _ => "AI/ML integration, Cloud-native development, Microservices, DevOps automation"
        };

        return $"Current trends in {domain}: {trends}";
    }
}

// Add functions to kernel
var businessFunctions = new BusinessFunctions();
kernel.ImportPluginFromObject(businessFunctions, "BusinessTools");

Console.WriteLine("✅ Custom business functions added to kernel");
Console.WriteLine("Functions available: CalculateProjectCost, GetTechnologyTrends");

## 9. Test Function Calling with Agents

Test how agents can discover and use the custom functions we've added.

In [ ]:
// Test agent with function calling capabilities
await TestSingleAgent("BusinessAnalyst", 
    "I need a cost estimate for a complex e-commerce project that will take 8 months with a team of 6 developers. " +
    "Also, what are the current technology trends for web development?");

## 10. Error Handling and Best Practices

Demonstrate proper error handling, logging, and security practices for production use.

In [ ]:
// Demonstrate production-ready error handling and security practices
public class SecureAgentService
{
    private readonly ILogger<SecureAgentService> _logger;
    private readonly Kernel _kernel;
    
    public SecureAgentService(ILogger<SecureAgentService> logger, Kernel kernel)
    {
        _logger = logger;
        _kernel = kernel;
    }
    
    public async Task<string> SafeAgentInteraction(string userInput, string agentName)
    {
        try
        {
            // Input validation and sanitization
            if (string.IsNullOrWhiteSpace(userInput))
            {
                throw new ArgumentException("User input cannot be empty");
            }
            
            if (userInput.Length > 2000)
            {
                throw new ArgumentException("User input exceeds maximum length");
            }
            
            // Rate limiting simulation
            await Task.Delay(100); // Simulate rate limiting
            
            _logger.LogInformation("Processing request for agent {AgentName}", agentName);
            
            // Secure agent execution with timeout
            using var cts = new CancellationTokenSource(TimeSpan.FromMinutes(2));
            
            if (!agents.ContainsKey(agentName))
            {
                throw new InvalidOperationException($"Agent {agentName} not found");
            }
            
            var agent = agents[agentName];
            var history = new ChatHistory();
            history.AddUserMessage(userInput);
            
            var response = new StringBuilder();
            await foreach (var chunk in agent.InvokeStreamingAsync(history, cancellationToken: cts.Token))
            {
                response.Append(chunk.Content);
            }
            
            _logger.LogInformation("Successfully processed request for agent {AgentName}", agentName);
            return response.ToString();
        }
        catch (OperationCanceledException)
        {
            _logger.LogWarning("Request timeout for agent {AgentName}", agentName);
            return "Request timed out. Please try again with a simpler query.";
        }
        catch (Exception ex)
        {
            _logger.LogError(ex, "Error processing request for agent {AgentName}", agentName);
            return "An error occurred while processing your request. Please try again.";
        }
    }
}

// Test secure service
var secureService = new SecureAgentService(
    loggerFactory.CreateLogger<SecureAgentService>(), 
    kernel);

var secureResponse = await secureService.SafeAgentInteraction(
    "What are the security considerations for cloud applications?", 
    "TechExpert");

Console.WriteLine($"Secure response: {secureResponse.Substring(0, Math.Min(200, secureResponse.Length))}...");
Console.WriteLine("\n✅ Demonstrated secure agent interaction with proper error handling");

## 11. Summary and Next Steps

Congratulations! You've successfully created a comprehensive .NET Semantic Kernel implementation with:

### ✅ What You've Built
- **Multi-agent AI system** using Semantic Kernel with Azure AI services
- **Secure authentication** with Azure OpenAI and Azure AI Inference
- **Agent group chat** with intelligent selection strategies
- **Custom function calling** for extended capabilities
- **Production-ready error handling** and security practices
- **Comprehensive logging** and monitoring

### 🚀 Next Steps
1. **Deploy to Azure**: Use Azure Container Apps or App Service
2. **Add more functions**: Integrate with your business systems
3. **Scale the solution**: Implement caching and rate limiting
4. **Monitor performance**: Add Application Insights
5. **Secure further**: Implement Managed Identity in production

### 📚 Resources
- [Semantic Kernel Documentation](https://learn.microsoft.com/semantic-kernel/)
- [Azure OpenAI Service](https://learn.microsoft.com/azure/ai-services/openai/)
- [Azure AI Inference SDK](https://learn.microsoft.com/azure/ai-studio/how-to/develop/sdk-overview)

### 🔗 Integration
This notebook complements the full workshop project available in the parent directories with FastAPI Python implementations and a React frontend.